In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import sklearn

In [3]:
sys.version

'3.9.7 (default, Sep 16 2021, 08:50:36) \n[Clang 10.0.0 ]'

In [4]:
print(os.getcwd())

/Users/christopherbutcher/Physics/propagation of interplanetary shocks/Welle Code


## package space
1) download the "space" package from git :
    git clone https://github.com/LaboratoryOfPlasmaPhysics/space
2) import the package (cf the cell below, where you may need to adapt the "path")

In [5]:
sys.path.append('./space')
import space
from space.models import planetary as smp
from space import smath as sm
from space.coordinates import coordinates as scc
from space import utils as su

ModuleNotFoundError: No module named 'space'

In [6]:
# magnetosheath model using shue98 et jelinek2012
msh = smp.Magnetosheath(magnetopause='mp_shue1998', bow_shock ='bs_jelinek2012')

In [7]:
# exemple d'utilisation de l'objet magnetogaine dont les arguments sont (theta,phi,kwargs) avec theta l'angle d'élevation par rapport à l'axe X et phi l'angle dans le cadrant YZ (phi=0 -> Y=0 et Z>0)
print('nose of the magnetopause :', msh.magnetopause(0,0))
print('\nnose of the bow shock :', msh.bow_shock(0,0))
# utilisation des kwargs pour changer les paramètres de vent solaire 
print('\nnose of the magnetopause for dynamic pressure of 3nPa :', msh.magnetopause(0,0,Pd=4))
print('\nnez of the bow shock for dynamic pressure of 3nPa :', msh.bow_shock(0,0,Pd=4))
# possibilité d'utiliser aves des array 1D ou 2D
print('\nmagnetopause terminator (x=0) in cartesian coordinates: ')
x,y,z = msh.magnetopause(np.pi/2, np.linspace(0,2*np.pi,20))
print('x = ', x,'\ny = ',y,'\nz = ',z)
print('\nmagnetopause terminator (x=0) in spherical coordinates: ')
r,theta,phi = msh.magnetopause(np.pi/2, np.linspace(0,2*np.pi,20),coord_sys='spherical')
print('r = ', r,'\ntheta = ',theta,'\nphi = ',phi)

nose of the magnetopause : (10.20902894053932, 0.0, 0.0)

nose of the bow shock : (13.454889791750555, 0.0, 0.0)

nose of the magnetopause for dynamic pressure of 3nPa : (9.22977189819985, 0.0, 0.0)

nez of the bow shock for dynamic pressure of 3nPa : (12.154927611159202, 0.0, 0.0)

magnetopause terminator (x=0) in cartesian coordinates: 
x =  9.409901333005365e-16 
y =  [ 0.00000000e+00  4.98983049e+00  9.43893542e+00  1.28651852e+01
  1.48972926e+01  1.53150472e+01  1.40731787e+01  1.13062630e+01
  7.31413827e+00  2.52941314e+00 -2.52941314e+00 -7.31413827e+00
 -1.13062630e+01 -1.40731787e+01 -1.53150472e+01 -1.48972926e+01
 -1.28651852e+01 -9.43893542e+00 -4.98983049e+00 -3.76396053e-15] 
z =  [ 15.3675351   14.53487966  12.12714447   8.40524502   3.77250684
  -1.26904099  -6.17306854 -10.40814832 -13.51534374 -15.15794195
 -15.15794195 -13.51534374 -10.40814832  -6.17306854  -1.26904099
   3.77250684   8.40524502  12.12714447  14.53487966  15.3675351 ]

magnetopause terminator (x=0

In [8]:
# charger le knn qui à été entrainer dans le systeme de coordonnée SWI avec K= 50 000 pts  :
interpvall = pd.read_pickle('KNN_swi_Vall_k50000.pkl')

# The KNN is only valid between the average models of Shue 1998 and Jelinek 2012. 

In [9]:
x_bow_shock = msh.bow_shock(0,0)[0]
x_magnetopause = msh.magnetopause(0,0)[0]

Dx_msh = x_bow_shock - x_magnetopause

In [10]:
# usage for a single point:
x,y,z = x_magnetopause + 1*Dx_msh, 0, 0
pos = np.asarray([[x,y,z]])
vx,vy,vz = interpvall.predict(pos).T
vx,vy,vz

(array([-0.21821791]), array([-0.02315195]), array([-0.01266387]))

In [11]:
v = sm.norm(vx,vy,vz) # calculation of the norm
print(' average of vmsh/vsw mid magnetsheath at the terminator  = ', v)

 average of vmsh/vsw mid magnetsheath at the terminator  =  [0.21980774]


In [12]:
# usage for several points in the middle of the magneotsheath at the terminator :
theta = np.pi/2 
phi = np.linspace(0,2*np.pi,100)
rbs = msh.bow_shock(theta,phi,coord_sys='spherical')[0]
rmp = msh.bow_shock(theta,phi,coord_sys='spherical')[0]
r= rmp + 0.5*(rbs-rmp) # radius corresponding to the middle of the magnetosheath
x,y,z = scc.spherical_to_cartesian(r,theta,phi)
x = np.zeros_like(y)+x #all the arrays must have the same dimension ( x = theta's dimension, here 1 , y and z have the dimension of phi, here 100 )
pos = np.asarray([x,y,z]).T # positions must always have shape (n,3)
vx,vy,vz = interpvall.predict(pos).T


In [13]:
v = sm.norm(vx,vy,vz) # calculation of the norm
print(' average of vmsh/vsw mid magnetsheath at the terminator  = ', np.mean(v))
print(' standard deviation of vmsh/vsw mid magnetsheath at the terminator  = ', np.std(v)) # Note that the magnetic field has a weird oriention in SWI so Qpara (Y>0) and Qperp (Y<0) do not have exactly the same velocity


 average of vmsh/vsw mid magnetsheath at the terminator  =  0.7106652764119145
 standard deviation of vmsh/vsw mid magnetsheath at the terminator  =  0.01228195581311484
